In [1]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import openai

openai.api_key = "Get Your Own API Key"



In [3]:
import pandas as pd

In [4]:
datafile_path  = "https://cdn.openai.com/API/examples/data/fine_food_reviews_with_embeddings_1k.csv"
df = pd.read_csv(datafile_path)

In [5]:
df.columns

Index(['ProductId', 'UserId', 'Score', 'Summary', 'Text', 'combined',
       'n_tokens', 'babbage_similarity', 'babbage_search'],
      dtype='object')

In [6]:
exp_df = df[["Summary","Text"]]

In [7]:
exp_df_mini = exp_df[:30]
exp_df_mini

,Summary,Text
0,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...
1,Arrived in pieces,"Not pleased at all. When I opened the box, mos..."
2,"It isn't blanc mange, but isn't bad . . .",I'm not sure that custard is really custard wi...
3,These also have SALT and it's not sea salt.,I like the fact that you can see what you're g...
4,Happy with the product,My dog was suffering with itchy skin. He had ...
5,Good Sauce,This is a good all purpose sauce. Has good fl...
6,Blackcat,Great coffee! Love all Green Mountain coffee ...
7,Excellent product,After scouring every store in town for orange ...
8,Bulk k-Cups,This is the best way to buy coffee for my offi...
9,It's Okay,"Next time, I will buy Gevalia Irish Cream deca..."


In [8]:
exp_df_mini["combined"] = "Title: "+exp_df_mini["Summary"]+"; Content: "+exp_df_mini["Text"]

<ipython-input-8-cdb2d83f406c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exp_df_mini["combined"] = "Title: "+exp_df_mini["Summary"]+"; Content: "+exp_df_mini["Text"]


In [9]:
similar_resp = openai.Embedding.create(input=exp_df_mini["combined"].to_list(),engine="text-similarity-davinci-001")

In [11]:
similar_vectors = []
for num in range(0,30):
  similar_vectors.append(similar_resp["data"][num]["embedding"])

In [12]:
search_resp = openai.Embedding.create(input=exp_df_mini["combined"].to_list(),engine="text-search-davinci-query-001")

In [13]:
search_vectors = []
for num in range(0,30):
  search_vectors.append(search_resp["data"][num]["embedding"])

In [14]:
exp_df_mini["search"] = search_vectors
exp_df_mini["similar"] = similar_vectors

<ipython-input-14-04b510612525>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exp_df_mini["search"] = search_vectors
<ipython-input-14-04b510612525>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exp_df_mini["similar"] = similar_vectors


In [17]:
from openai.embeddings_utils import get_embedding, cosine_similarity
def search_reviews(df, search_query, n=3):
    embedding = get_embedding(
        search_query,
        engine="text-search-davinci-query-001"
    )
    df["similarities"] = df.search.apply(lambda x: cosine_similarity(x, embedding))

    top_n =df.sort_values("similarities", ascending=False).head(n)
    return top_n

In [18]:
search_reviews(exp_df_mini,"good coffee")

<ipython-input-17-9d5a7367075d>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["similarities"] = df.search.apply(lambda x: cosine_similarity(x, embedding))


,Summary,Text,combined,search,similar,similarities
6,Blackcat,Great coffee! Love all Green Mountain coffee ...,Title: Blackcat; Content: Great coffee! Love ...,"[0.012105066329240799, -0.011309935711324215, ...","[-0.008968373760581017, 0.0013170117745175958,...",0.553884
23,Great flavor no bite,This coffee is a favorite of mine and many oth...,Title: Great flavor no bite; Content: This cof...,"[0.007149829529225826, -0.012162343598902225, ...","[-0.0030176059808582067, 0.0023373363073915243...",0.532972
21,Great flavor no bite,This coffee is a favorite of mine and many oth...,Title: Great flavor no bite; Content: This cof...,"[0.008452225476503372, -0.011746622622013092, ...","[-0.003538617165759206, 0.0029234811663627625,...",0.528752


In [19]:
from openai.embeddings_utils import get_embedding, cosine_similarity
def search_reviews_sim(df, search_query, n=3):
    embedding = get_embedding(
        search_query,
        engine="text-search-davinci-query-001"
    )
    df["similarities"] = df.similar.apply(lambda x: cosine_similarity(x, embedding))

    top_n =df.sort_values("similarities", ascending=False).head(n)
    return top_n

In [20]:
search_reviews_sim(exp_df_mini,"good coffee")

<ipython-input-19-34b00f1fbfc0>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["similarities"] = df.similar.apply(lambda x: cosine_similarity(x, embedding))


,Summary,Text,combined,search,similar,similarities
21,Great flavor no bite,This coffee is a favorite of mine and many oth...,Title: Great flavor no bite; Content: This cof...,"[0.008452225476503372, -0.011746622622013092, ...","[-0.003538617165759206, 0.0029234811663627625,...",0.320315
6,Blackcat,Great coffee! Love all Green Mountain coffee ...,Title: Blackcat; Content: Great coffee! Love ...,"[0.012105066329240799, -0.011309935711324215, ...","[-0.008968373760581017, 0.0013170117745175958,...",0.319496
23,Great flavor no bite,This coffee is a favorite of mine and many oth...,Title: Great flavor no bite; Content: This cof...,"[0.007149829529225826, -0.012162343598902225, ...","[-0.0030176059808582067, 0.0023373363073915243...",0.318418
